##1. Get authenticated from tweeter API

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/NLP'

apikey= 'du9vEcqQFCtdu9vEcqQFCt'
apisecretkey= 'w7JsH0E4KCfOTMdcEw7JsH0E4KCfOTMdcE'
accesstoken='126864770-A5bLHXKcDFFk126864770-A5bLHXKcDFFk'
accesstokensecret='JS1l5hXn9dSM8lfJS1l5hXn9dSM8lf'

import os
import tweepy as tw

auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)

api = tw.API(auth,wait_on_rate_limit=True)

Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks/NLP


In [2]:
dir(api)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_list_members',
 '_lookup_friendships',
 '_pack_image',
 '_remove_list_members',
 '_send_direct_message',
 'add_list_member',
 'add_list_members',
 'api_root',
 'auth',
 'blocks',
 'blocks_ids',
 'cache',
 'compression',
 'configuration',
 'create_block',
 'create_favorite',
 'create_friendship',
 'create_list',
 'create_media_metadata',
 'create_mute',
 'create_saved_search',
 'destroy_block',
 'destroy_direct_message',
 'destroy_favorite',
 'destroy_friendship',
 'destroy_list',
 'destroy_mute',
 'destroy_saved_search',
 'destroy_status',
 'favorites',
 'followers',
 'followers_ids',
 'friends',
 'friends_ids',
 'f

#2: Know what's on my timeline

In [3]:
my_timeline = api.home_timeline()
for status in my_timeline:
  #print(status.text)
  print(vars(status))

{'_api': <tweepy.api.API object at 0x7f3ab736d890>, '_json': {'created_at': 'Wed Oct 26 07:30:03 +0000 2022', 'id': 1585171884773089280, 'id_str': '1585171884773089280', 'text': 'डेंगू होने पर तेज बुखार आता है l लक्षण होने पर तुरंत डॉक्टर से मिलें l\n@BiharHealthDept \n@SHSBihar \n#डेंगू\n#Dengue… https://t.co/T6ItXDVb1z', 'truncated': True, 'entities': {'hashtags': [{'text': 'डेंगू', 'indices': [100, 106]}, {'text': 'Dengue', 'indices': [107, 114]}], 'symbols': [], 'user_mentions': [{'screen_name': 'BiharHealthDept', 'name': 'Bihar Health Dept', 'id': 1135503797025828864, 'id_str': '1135503797025828864', 'indices': [71, 87]}, {'screen_name': 'SHSBihar', 'name': 'State Health Society Bihar', 'id': 1242380809064968192, 'id_str': '1242380809064968192', 'indices': [89, 98]}], 'urls': [{'url': 'https://t.co/T6ItXDVb1z', 'expanded_url': 'https://twitter.com/i/web/status/1585171884773089280', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [116, 139]}]}, 'source': '<a href="https://

In [4]:
columns = set()
allowed_types = [str,int]
tweets_data = []
for status in my_timeline:
  status_dict = dict(vars(status))
  keys = vars(status).keys()
  single_tweet_data = {'user':status.user.screen_name, 'author':status.author.screen_name}
  for k in keys:
    try:
      #print(k)
      v_type=type(status_dict[k])
    except:
      v_type = None
    if v_type != None:

        if v_type in allowed_types:
          single_tweet_data[k] = status_dict[k]
          columns.add(k)

  tweets_data.append(single_tweet_data)
header_cols = list(columns)

header_cols.append('user')
header_cols.append('author')
header_cols

['retweet_count',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'id_str',
 'text',
 'lang',
 'quoted_status_id_str',
 'in_reply_to_screen_name',
 'id',
 'source_url',
 'quoted_status_id',
 'in_reply_to_user_id_str',
 'favorite_count',
 'in_reply_to_user_id',
 'source',
 'user',
 'author']

In [5]:
tweets_data

[{'user': 'IPRD_Bihar',
  'author': 'IPRD_Bihar',
  'id': 1585171884773089280,
  'id_str': '1585171884773089280',
  'text': 'डेंगू होने पर तेज बुखार आता है l लक्षण होने पर तुरंत डॉक्टर से मिलें l\n@BiharHealthDept \n@SHSBihar \n#डेंगू\n#Dengue… https://t.co/T6ItXDVb1z',
  'source': 'Meltwater Social',
  'source_url': 'https://www.meltwater.com/',
  'retweet_count': 1,
  'favorite_count': 3,
  'lang': 'hi'},
 {'user': 'IPRD_Bihar',
  'author': 'IPRD_Bihar',
  'id': 1585170264568266754,
  'id_str': '1585170264568266754',
  'text': 'माननीय मुख्यमंत्री @Nitishkumar ने विधायक आवासन योजना अन्तर्गत माननीय विधायकगण हेतु निर्मित आवासों का उद्घाटन किया… https://t.co/A8bmxJve3U',
  'source': 'Twitter Web App',
  'source_url': 'https://mobile.twitter.com',
  'retweet_count': 0,
  'favorite_count': 7,
  'lang': 'hi'},
 {'user': 'editorji',
  'author': 'editorji',
  'id': 1585169917716074496,
  'id_str': '1585169917716074496',
  'text': 'LIVE | Harry Brook and Dawid Malan rebuild after early wickets

In [6]:
import pandas as pd
df = pd.DataFrame(tweets_data,columns = header_cols)
df.head()

,retweet_count,in_reply_to_status_id,in_reply_to_status_id_str,id_str,text,lang,quoted_status_id_str,in_reply_to_screen_name,id,source_url,quoted_status_id,in_reply_to_user_id_str,favorite_count,in_reply_to_user_id,source,user,author
0,1,NaN,NaN,1585171884773089280,डेंगू होने पर तेज बुखार आता है l लक्षण होने पर...,hi,NaN,NaN,1585171884773089280,https://www.meltwater.com/,NaN,NaN,3,NaN,Meltwater Social,IPRD_Bihar,IPRD_Bihar
1,0,NaN,NaN,1585170264568266754,माननीय मुख्यमंत्री @Nitishkumar ने विधायक आवास...,hi,NaN,NaN,1585170264568266754,https://mobile.twitter.com,NaN,NaN,7,NaN,Twitter Web App,IPRD_Bihar,IPRD_Bihar
2,0,1.585165e+18,1585165443299672064,1585169917716074496,LIVE | Harry Brook and Dawid Malan rebuild aft...,en,NaN,editorji,1585169917716074496,https://mobile.twitter.com,NaN,896740409346371584,0,8.967404e+17,Twitter Web App,editorji,editorji
3,10,NaN,NaN,1585169356627275776,J&amp;K | Encounter started at Wanseeran Tarip...,en,NaN,NaN,1585169356627275776,https://mobile.twitter.com,NaN,NaN,54,NaN,Twitter Web App,ANI,ANI
4,16,NaN,NaN,1585169295582973952,"Their (AAP) minister, Gujarat chief &amp; lead...",en,1585146256057602050,NaN,1585169295582973952,https://mobile.twitter.com,1.585146e+18,NaN,67,NaN,Twitter Web App,ANI,ANI


#3: What's on different user's timeline

In [7]:
def extract_timeline_as_df(timeline_list):
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    for status in timeline_list:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except:
                v_type = None
            if v_type != None:
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)


    header_cols = list(columns)
    header_cols.append("user")
    header_cols.append('author')
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [8]:
my_timeline = api.home_timeline()

In [9]:
df2 = extract_timeline_as_df(my_timeline)
df2.head()

,retweet_count,in_reply_to_status_id,in_reply_to_status_id_str,id_str,text,lang,quoted_status_id_str,in_reply_to_screen_name,id,source_url,quoted_status_id,in_reply_to_user_id_str,favorite_count,in_reply_to_user_id,source,user,author
0,1,NaN,NaN,1585171884773089280,डेंगू होने पर तेज बुखार आता है l लक्षण होने पर...,hi,NaN,NaN,1585171884773089280,https://www.meltwater.com/,NaN,NaN,3,NaN,Meltwater Social,IPRD_Bihar,IPRD_Bihar
1,0,NaN,NaN,1585170264568266754,माननीय मुख्यमंत्री @Nitishkumar ने विधायक आवास...,hi,NaN,NaN,1585170264568266754,https://mobile.twitter.com,NaN,NaN,7,NaN,Twitter Web App,IPRD_Bihar,IPRD_Bihar
2,0,1.585165e+18,1585165443299672064,1585169917716074496,LIVE | Harry Brook and Dawid Malan rebuild aft...,en,NaN,editorji,1585169917716074496,https://mobile.twitter.com,NaN,896740409346371584,0,8.967404e+17,Twitter Web App,editorji,editorji
3,10,NaN,NaN,1585169356627275776,J&amp;K | Encounter started at Wanseeran Tarip...,en,NaN,NaN,1585169356627275776,https://mobile.twitter.com,NaN,NaN,54,NaN,Twitter Web App,ANI,ANI
4,16,NaN,NaN,1585169295582973952,"Their (AAP) minister, Gujarat chief &amp; lead...",en,1585146256057602050,NaN,1585169295582973952,https://mobile.twitter.com,1.585146e+18,NaN,67,NaN,Twitter Web App,ANI,ANI


In [10]:
user = api.get_user("sudhirchaudhary")
user_timeline = user.timeline()
df3 = extract_timeline_as_df(user_timeline)
df3.head()

,retweet_count,id_str,text,lang,quoted_status_id_str,id,source_url,quoted_status_id,favorite_count,source,user,author
0,144,1585161659655819264,RT @ratingology: 𝐒𝐭𝐚𝐭𝐬 𝐨𝐟 𝐥𝐢𝐯𝐞 𝐯𝐢𝐞𝐰𝐞𝐫𝐬 𝐨𝐧 #You...,en,NaN,1585161659655819264,http://twitter.com/download/iphone,NaN,0,Twitter for iPhone,sudhirchaudhary,sudhirchaudhary
1,49,1584957124601602049,If you missed tonight’s Black&amp;White you ca...,en,1584929004154654720,1584957124601602049,http://twitter.com/download/iphone,1.584929e+18,364,Twitter for iPhone,sudhirchaudhary,sudhirchaudhary
2,37,1584952291471544320,RT @aajtak: ब्रिटेन के पूर्व प्रधानमंत्रियों क...,hi,NaN,1584952291471544320,http://twitter.com/download/iphone,NaN,0,Twitter for iPhone,sudhirchaudhary,sudhirchaudhary
3,87,1584952274664976384,RT @aajtak: ऋषि सुनक को कुर्सी धर्म से मिली या...,hi,NaN,1584952274664976384,http://twitter.com/download/iphone,NaN,0,Twitter for iPhone,sudhirchaudhary,sudhirchaudhary
4,26,1584952246588276737,RT @aajtak: ऋषि सुनक के भारतीय मूल पर जश्न मना...,hi,NaN,1584952246588276737,http://twitter.com/download/iphone,NaN,0,Twitter for iPhone,sudhirchaudhary,sudhirchaudhary


#4: Reply any tweet 

In [11]:
# og_tweet = api.get_status("1584907651753746432")
# # print(og_tweet.user.screen_name, og_tweet.id)
# my_reply = api.update_status(f"@{og_tweet.user.screen_name} Wow this cool!", og_tweet.id)
#print(my_reply.id, my_reply.user.screen_name)

#5: Find all followers and friends count

In [12]:
user = api.get_user("ipvikas")
print(user.followers_count, user.friends_count)

29 209


In [13]:
user_friends = user.friends()
for friend in user_friends:
  print(friend.screen_name)

ideasforhom
ilyasut
OriolVinyalsML
misraturp
keithgalli
ForrestPKnight
HouseofDragon
IPS_SantoshM
DrSJaishankar
cop_anirudha
Rapid_API
ANI
DOITBihar
ykdojo
KenJee_DS
PrasoonPratham
DataChaz
philipvollet
airnewsalerts
CausalSandeep


#6: Items & Pagination with Tweepy Cursor

In [14]:
len(api.home_timeline(count=20))

20

In [15]:
import tweepy

user = api.get_user("sudhirchaudhary")

for i, status in enumerate(tweepy.Cursor(api.home_timeline, count=50).items(50)):
    print(i, status.text)

0 डेंगू होने पर तेज बुखार आता है l लक्षण होने पर तुरंत डॉक्टर से मिलें l
@BiharHealthDept 
@SHSBihar 
#डेंगू
#Dengue… https://t.co/T6ItXDVb1z
1 माननीय मुख्यमंत्री @Nitishkumar ने विधायक आवासन योजना अन्तर्गत माननीय विधायकगण हेतु निर्मित आवासों का उद्घाटन किया… https://t.co/A8bmxJve3U
2 LIVE | Harry Brook and Dawid Malan rebuild after early wickets. England 63-3 after 10 overs. 

Check #ENGvIRE live… https://t.co/y6usbfl1Y3
3 J&amp;K | Encounter started at Wanseeran Taripora area of Baramulla. Police and security forces are on the job. Further… https://t.co/edcK73wJpZ
4 Their (AAP) minister, Gujarat chief &amp; leaders have abused Hindu gods &amp; said many things &amp; yet, they are in the part… https://t.co/llh9mswMuJ
5 Southsea pizza vending machine plan withdrawn.
https://t.co/X0d8KZgbVS
6 RT @ani_digital: 'Normalization of India, China ties in world's interest': Jaishankar after China envoy's farewell call 

Read @ANI Story |…
7 Only at #TechSparks2022, the destination for deliberat

#7: Search for any tweet from any start date

In [16]:
search_words = ["#SuryaGrahan"] # ["#ipl2020","#DCvKXIP"]; ["#coronavirus","#Modi"]; ["#TwinTowers"] 
date_since = "2022-10-25"

In [17]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(10)

tweets

#will take some time 2 min
# tweet_details = [[tweet.geo, tweet.text,tweet.user.screen_name, tweet.user.location] for tweet in tweets]
tweet_details = [[tweet.geo, tweet.text,tweet.user.screen_name, tweet.user.location,tweet.created_at,[e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count] for tweet in tweets]
tweet_details

[[None,
  "RT @news18dotcom: As a partial #solareclipse is taking place today, here's a look at things to know about this astronomical event.\n\n#SolarE…",
  'DavoriyaAmitabh',
  '',
  datetime.datetime(2022, 10, 26, 7, 6, 33),
  ['solareclipse'],
  26],
 [None,
  'RT @StringReveals: No scarcity of idiots who ridicule sages but I assure you will reep immense benefits if you *skip your lunch today*👍\n-Av…',
  'anas90098573',
  '',
  datetime.datetime(2022, 10, 26, 6, 57, 56),
  [],
  19],
 [None,
  'RT @SaffronDelhite: So True 😂🤭\n\n#SolarEclipse2022 #SuryaGrahan #SolarEclipse https://t.co/IyTo3ZhY0m',
  'ItsMeSiri0831',
  '',
  datetime.datetime(2022, 10, 26, 6, 38, 49),
  ['SolarEclipse2022', 'SuryaGrahan', 'SolarEclipse'],
  28],
 [None,
  'RT @QGroupMedia7200: Live : సూర్య గ్రహణం || Solar Eclipse Live || TeenmarMallanna || QNews || QNewsHD https://t.co/omqRs6gUur @TeenmarMalla…',
  'Nagesh52200867',
  '',
  datetime.datetime(2022, 10, 26, 6, 37, 13),
  [],
  16],
 [None,
  'RT @Saff

In [18]:
import pandas as pd
tweet_df = pd.DataFrame(data=tweet_details, columns=['geo','text','user', "location","Timestamp","All_hashtags","Followers_Count"])
pd.set_option('max_colwidth', 80)
tweet_df.head(5)

,geo,text,user,location,Timestamp,All_hashtags,Followers_Count
0,None,"RT @news18dotcom: As a partial #solareclipse is taking place today, here's a...",DavoriyaAmitabh,,2022-10-26 07:06:33,[solareclipse],26
1,None,RT @StringReveals: No scarcity of idiots who ridicule sages but I assure you...,anas90098573,,2022-10-26 06:57:56,[],19
2,None,RT @SaffronDelhite: So True 😂🤭\n\n#SolarEclipse2022 #SuryaGrahan #SolarEclip...,ItsMeSiri0831,,2022-10-26 06:38:49,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",28
3,None,RT @QGroupMedia7200: Live : సూర్య గ్రహణం || Solar Eclipse Live || TeenmarMal...,Nagesh52200867,,2022-10-26 06:37:13,[],16
4,None,RT @SaffronDelhite: So True 😂🤭\n\n#SolarEclipse2022 #SuryaGrahan #SolarEclip...,HimanshuNayakk,India,2022-10-26 06:36:38,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",149


In [19]:
tweet_df.user.value_counts()

DavoriyaAmitabh    1
anas90098573       1
ItsMeSiri0831      1
Nagesh52200867     1
HimanshuNayakk     1
ArunTikmani        1
haramibabuji       1
Elevennn___        1
_i_0_absolute      1
TariqMa17497904    1
Name: user, dtype: int64

In [20]:
tweet_df.location.value_counts()

                          6
India                     1
Bhubaneswar, India        1
Mirzapur, Afghanistan     1
india                     1
Name: location, dtype: int64

In [21]:

# emoticons_happy = set([
#     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
#     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
#     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
#     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
#     '<3'
#     ])
 

# emoticons_sad = set([
#     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
#     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
#     ':c', ':{', '>:\\', ';('
#     ])
 

# emoticons = emoticons_happy.union(emoticons_sad)

# import re
# def clean_tweets(text):
#     text = re.sub("RT @[\w]*:","",text)
#     text = re.sub("@[\w]*","",text)
#     text = re.sub("https?://[A-Za-z0-9./]*","",text)
#     text = re.sub("\n","",text)


#     text = re.sub("\$\w*","",text)
#     text = re.sub("^RT[\s]+","",text)
#     text = re.sub("https?:\/\/.*[\r\n]*","",text)
#     text = re.sub("#","",text)


#     tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#     tweet_tokens = tokenizer.tokenize(text)
 
#     tweets_clean = []    
#     for word in tweet_tokens:
#         if (word not in stopwords_english and
#               word not in emoticons and 
#                 word not in string.punctuation): 
           
#             stem_word = stemmer.stem(word)
#             tweets_clean.append(stem_word)
 
#     return tweets_clean
#     text = tweets_clean 
 



#     # return text

# tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

# tweet_df.head(20)
# tweet_df.to_csv('tweets.csv')
# # !ls



# # tweet = re.sub(r'\$\w*', '', tweet)
# # tweet = re.sub(r'^RT[\s]+', '', tweet)
# # tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
# # tweet = re.sub(r'#', '', tweet)

In [22]:
import re
def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",text)
    text = re.sub("@[\w]*","",text)
    text = re.sub("https?://[A-Za-z0-9./]*","",text)
    text = re.sub("\n","",text)


    text = re.sub("\$\w*","",text)
    text = re.sub("^RT[\s]+","",text)
    text = re.sub("https?:\/\/.*[\r\n]*","",text)
    text = re.sub("#","",text)

    return text

tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

tweet_df.head(20)
tweet_df.to_csv('tweets.csv')
# !ls

In [23]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [24]:
tweet_df['text'].apply(lambda x: [print("\tText : {}, Entity : {}".format(ent.text, ent.label_)) if (not ent.text.startswith('#')) else ""  for ent in nlp(x).ents])

	Text : today, Entity : DATE
	Text : SolarE…, Entity : ORG
	Text : The " S ", Entity : WORK_OF_ART
	Text : SuryaGrahan, Entity : ORG
	Text : SuryaGrahan, Entity : ORG
	Text : India, Entity : GPE
	Text : Eclipse, Entity : PRODUCT


0          [None, None]
1                    []
2                    []
3                    []
4                    []
5                    []
6                    []
7          [None, None]
8                    []
9    [None, None, None]
Name: text, dtype: object

In [25]:
tweet_df['entities']=tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])
tweet_df.head(20)

,geo,text,user,location,Timestamp,All_hashtags,Followers_Count,entities
0,None,"As a partial solareclipse is taking place today, here's a look at things to...",DavoriyaAmitabh,,2022-10-26 07:06:33,[solareclipse],26,"[(today, DATE), (SolarE…, ORG)]"
1,None,No scarcity of idiots who ridicule sages but I assure you will reep immense...,anas90098573,,2022-10-26 06:57:56,[],19,[]
2,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,ItsMeSiri0831,,2022-10-26 06:38:49,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",28,[]
3,None,Live : సూర్య గ్రహణం || Solar Eclipse Live || TeenmarMallanna || QNews || QN...,Nagesh52200867,,2022-10-26 06:37:13,[],16,[]
4,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,HimanshuNayakk,India,2022-10-26 06:36:38,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",149,[]
5,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,ArunTikmani,"Bhubaneswar, India",2022-10-26 06:32:04,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",569,[]
6,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,haramibabuji,"Mirzapur, Afghanistan",2022-10-26 06:24:01,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",496,[]
7,None,"The "" S "" in solar eclipse stands for ✨ SERENE ✨SolarEclipse2022 SuryaGrahan",Elevennn___,,2022-10-26 06:19:11,"[SolarEclipse2022, SuryaGrahan]",3034,"[(The "" S "", WORK_OF_ART), (SuryaGrahan, ORG)]"
8,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,_i_0_absolute,india,2022-10-26 06:18:59,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",118,[]
9,None,📸Partial Solar eclipse SuryaGrahan 🌞as observed in India. Solar Eclipse occ...,TariqMa17497904,,2022-10-26 06:15:44,[SuryaGrahan],2,"[(SuryaGrahan, ORG), (India, GPE), (Eclipse, PRODUCT)]"


In [26]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

tweet_df['sentiment']=tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [27]:
tweet_df.head(10)

,geo,text,user,location,Timestamp,All_hashtags,Followers_Count,entities,sentiment
0,None,"As a partial solareclipse is taking place today, here's a look at things to...",DavoriyaAmitabh,,2022-10-26 07:06:33,[solareclipse],26,"[(today, DATE), (SolarE…, ORG)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
1,None,No scarcity of idiots who ridicule sages but I assure you will reep immense...,anas90098573,,2022-10-26 06:57:56,[],19,[],"{'neg': 0.138, 'neu': 0.613, 'pos': 0.249, 'compound': 0.5994}"
2,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,ItsMeSiri0831,,2022-10-26 06:38:49,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",28,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}"
3,None,Live : సూర్య గ్రహణం || Solar Eclipse Live || TeenmarMallanna || QNews || QN...,Nagesh52200867,,2022-10-26 06:37:13,[],16,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
4,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,HimanshuNayakk,India,2022-10-26 06:36:38,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",149,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}"
5,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,ArunTikmani,"Bhubaneswar, India",2022-10-26 06:32:04,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",569,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}"
6,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,haramibabuji,"Mirzapur, Afghanistan",2022-10-26 06:24:01,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",496,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}"
7,None,"The "" S "" in solar eclipse stands for ✨ SERENE ✨SolarEclipse2022 SuryaGrahan",Elevennn___,,2022-10-26 06:19:11,"[SolarEclipse2022, SuryaGrahan]",3034,"[(The "" S "", WORK_OF_ART), (SuryaGrahan, ORG)]","{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'compound': 0.5766}"
8,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,_i_0_absolute,india,2022-10-26 06:18:59,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",118,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}"
9,None,📸Partial Solar eclipse SuryaGrahan 🌞as observed in India. Solar Eclipse occ...,TariqMa17497904,,2022-10-26 06:15:44,[SuryaGrahan],2,"[(SuryaGrahan, ORG), (India, GPE), (Eclipse, PRODUCT)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"


In [28]:
!pip install googlemaps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=36aa8b6f0b0a7e1832babdf6a826586f5fe6bb140726d34d1965dda6460a3684
  Stored in directory: /root/.cache/pip/wheels/80/db/c0/6d958585fa97b20e250bf437acf7e6e715b4809c2dd4e55367
Successfully built googlemaps


In [29]:
import googlemaps

In [30]:
def get_country(input):
  try:
    output=gmaps.geocode(input)[0]['formatted_address'].split(",")[-1].strip()
  except:
    output="Error"
  return output

tweet_df['country']=tweet_df['location'].apply(lambda x: "" if (not x.strip()) else get_country(x))

tweet_df['country'].value_counts()

         6
Error    4
Name: country, dtype: int64

In [31]:
tweet_df.head(10)

,geo,text,user,location,Timestamp,All_hashtags,Followers_Count,entities,sentiment,country
0,None,"As a partial solareclipse is taking place today, here's a look at things to...",DavoriyaAmitabh,,2022-10-26 07:06:33,[solareclipse],26,"[(today, DATE), (SolarE…, ORG)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",
1,None,No scarcity of idiots who ridicule sages but I assure you will reep immense...,anas90098573,,2022-10-26 06:57:56,[],19,[],"{'neg': 0.138, 'neu': 0.613, 'pos': 0.249, 'compound': 0.5994}",
2,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,ItsMeSiri0831,,2022-10-26 06:38:49,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",28,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}",
3,None,Live : సూర్య గ్రహణం || Solar Eclipse Live || TeenmarMallanna || QNews || QN...,Nagesh52200867,,2022-10-26 06:37:13,[],16,[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",
4,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,HimanshuNayakk,India,2022-10-26 06:36:38,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",149,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}",Error
5,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,ArunTikmani,"Bhubaneswar, India",2022-10-26 06:32:04,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",569,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}",Error
6,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,haramibabuji,"Mirzapur, Afghanistan",2022-10-26 06:24:01,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",496,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}",Error
7,None,"The "" S "" in solar eclipse stands for ✨ SERENE ✨SolarEclipse2022 SuryaGrahan",Elevennn___,,2022-10-26 06:19:11,"[SolarEclipse2022, SuryaGrahan]",3034,"[(The "" S "", WORK_OF_ART), (SuryaGrahan, ORG)]","{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'compound': 0.5766}",
8,None,So True 😂🤭SolarEclipse2022 SuryaGrahan SolarEclipse,_i_0_absolute,india,2022-10-26 06:18:59,"[SolarEclipse2022, SuryaGrahan, SolarEclipse]",118,[],"{'neg': 0.0, 'neu': 0.564, 'pos': 0.436, 'compound': 0.4754}",Error
9,None,📸Partial Solar eclipse SuryaGrahan 🌞as observed in India. Solar Eclipse occ...,TariqMa17497904,,2022-10-26 06:15:44,[SuryaGrahan],2,"[(SuryaGrahan, ORG), (India, GPE), (Eclipse, PRODUCT)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",


In [32]:
tweet_df.dtypes

geo                        object
text                       object
user                       object
location                   object
Timestamp          datetime64[ns]
All_hashtags               object
Followers_Count             int64
entities                   object
sentiment                  object
country                    object
dtype: object

In [33]:
tweet_df.to_csv("tweets.csv")

In [34]:
#BASIC 1
import csv
import random
from nltk.corpus import twitter_samples as ts
import tweepy
#in anaconda prompt: pip install tweepy
from textblob import TextBlob

consumer_key= 'du9vEcqQFCtidu9vEcqQFCti'
consumer_secret= 'w7JsH0E4KCfOTMdcw7JsH0E4KCfOTMdc'

access_token='126864770-A126864770-A'
access_token_secret='JS1l5hXn9JS1l5hXn9'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#Step 3 - Retrieve Tweets
public_tweets = api.search('SuryaGrahan')

for tweet in public_tweets:
    print(tweet.text)
    
    #Step 4 Perform Sentiment Analysis on Tweets
    analysis = TextBlob(tweet.text)
    print(analysis.sentiment)
    print("")

RT @SatlokChannel: #SolarEclipse | वर्ष 2022 का आखिरी सूर्यग्रहण आज लग रहा है। शाम 4 बजे के पश्चात से इसे भिन्न बिन स्थानों से भिन्न समय…
Sentiment(polarity=0.0, subjectivity=0.0)

RT @SatlokChannel: #SolarEclipse | वर्ष 2022 का आखिरी सूर्यग्रहण आज लग रहा है। शाम 4 बजे के पश्चात से इसे भिन्न बिन स्थानों से भिन्न समय…
Sentiment(polarity=0.0, subjectivity=0.0)

Surya Grahan से प्रभावित हुए देश के प्रसिद्ध व प्राचीन मंदिर, 14 घंटे बाद खुले खजराना गणेश मंदिर के पट… https://t.co/1l0vrfGGEx
Sentiment(polarity=0.0, subjectivity=0.0)

RT @ZeeNews: चश्मा लगाकर गोरखपुर से  CM योगी  ने देखा सूर्यग्रहण 

#YogiAdityanath #solareclipse #SuryaGrahan @Chandans_live https://t.co/M…
Sentiment(polarity=0.0, subjectivity=0.0)

RT @News18UP: सूर्य ग्रहण देखने पहुंचे सीएम योगी 

#suryagrahan2022  #SuryaGrahan https://t.co/AZg2jrcFQB
Sentiment(polarity=0.0, subjectivity=0.0)

RT @MiSagarShalar: 🌞सुर्यग्रहण, नाशिक

२५ अॅाक्टोंबर २०२२ ५.३३ वा.

#suryagrahan #nashik #नाशिक #सुर्यग्रहण #mynashik https://t.co/o87

In [35]:
#BASIC 2: we have used it in above code
# https://github.com/ritvikmath/ScrapingData
import json
import csv
import tweepy
#in anaconda prompt: pip install tweepy

import re

"""
INPUTS:
    consumer_key, consumer_secret, access_token, access_token_secret: codes 
    telling twitter that we are authorized to access this data
    hashtag_phrase: the combination of hashtags to search for
OUTPUTS:
    none, simply save the tweet info to a spreadsheet
"""

def search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase):
    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #initialize Tweepy API
    api = tweepy.API(auth)
    
    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))

    #open the spreadsheet we will write to
    with open('%s 2_tweets.csv' % (fname), 'w') as file:
        w = csv.writer(file)

        #write header row to spreadsheet
        w.writerow(['timestamp', 'tweet_text', 'username', 'all_hashtags', 'followers_count'])

        #for each tweet matching our hashtags, write relevant info to the spreadsheet
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', lang="en", tweet_mode='extended').items(100):
            w.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count])
            
# consumer_key = input('Consumer Key')
# consumer_secret = input('Consumer Secret')
# access_token = input('Access Token')
# access_token_secret = input('Access Token Secret')
consumer_key= 'du9vEcqQFCtiiNSJYpDhjJwEW'
consumer_secret= 'w7JsH0E4KCfOTMdcEtSKa3RSo8CImQVlDW6cSeujpBT2mKTigS'
access_token='126864770-A5bLHXKcDFFkLqxFG296m56xlvGzskw4WMTIfRq5'
access_token_secret='JS1l5hXn9dSM8lf2TCsX6g8JpLzFA5DyQ2znHGvgNr1rD'

hashtag_phrase = input('Hashtag Phrase')

if __name__ == '__main__':
    search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase)

Hashtag PhraseSuryaGrahan


## 2. Scrape Twitter Data or Tweets using snscrape module

In [36]:
!pip install -q snscrape

import os
import pandas as pd
from datetime import date, timedelta

In [37]:
today = date.today()
end_date = today

search_term = 'SuryaGrahan'
from_date = date.today() - timedelta(days=1)# '2022-08-28'

####Total Number of Tweets for Search Terms

In [38]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/NLP'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/NLP


In [39]:
import os
# from_date = '2021-08-15'
# search_term = '#IndiaIndependenceDay'
# end_date = '2021-08-16'

os.system(f"snscrape --since {from_date} twitter-search '{search_term} until:{end_date}' > result-tweets.txt") #will take more than 10 minutes
if os.stat("result-tweets.txt").st_size == 0:
  counter = 0
else:
  df = pd.read_csv('result-tweets.txt', names=['link'])
  counter = df.size

print('Number Of Tweets : '+ str(counter))

Number Of Tweets : 1867


#####Extracting Exact Tweeets

In [40]:
max_results = 100

In [41]:
extracted_tweets = "snscrape --format '{content!r}'"+ f" --max-results {max_results} --since {from_date} twitter-search '{search_term} until:{end_date}' > extracted-tweets.txt"

# extracted_tweets = (re.sub('@[a-z,_,A-Z,0-9,:]+ ', '', i) for i in extracted_tweets)

os.system(extracted_tweets)
if os.stat("extracted-tweets.txt").st_size == 0:
  print('No Tweets found')
else:
  df = pd.read_csv('extracted-tweets.txt', names=['content'])
  for row in df['content'].iteritems():
    print(row)

(0, "'#SolarEclipse | वर्ष 2022 का आखिरी सूर्यग्रहण आज लग रहा है। शाम 4 बजे के पश्चात से इसे भिन्न बिन स्थानों से भिन्न समय में देखना संभव हो सकेगा। जानें सूर्य ग्रहण से संबंधित वर्जनाओं के खण्डन के बारे में।\\n\\n#SuryaGrahan #सूर्यग्रहण\\n\\nजानने के लिए पढ़ें:👉 https://t.co/bCJejtZYoM https://t.co/2IQ8GkjgZO'")
(1, "'@AmyMek \\n@AshleaSimonBF\\n·\\nplight f #hindus\\nMuslim PM **k of\\nSuryagrahan https://t.co/vn76spvvqy'")
(2, "'@_KritiGupta Yahi to asli walaa suryagrahan hai'")
(3, "'گرہن\\n The last solar eclipse of 2022\\n(25th Oct")
(4, "'#Watch | சூரிய கிரகணத்தை அடுத்து கும்பகோணத்தில் உள்ள திருநாகேஸ்வரம் கோயில் கதவை அடைத்து போராட்டத்தில் ஈடுபட்ட இந்து அமைப்பை சேர்ந்த 13 பேர் கைது \\n\\n#SunNews | #SuryaGrahan | #Thirunageswaram https://t.co/fCkeCDPiYv'")
(5, "'#Watch |  சூரிய கிரகணத்தால் செக்கச்சிவந்த வானம்!\\n\\n#SunNews | #SuryaGrahan | #SuryaGrahanam https://t.co/SkJaQ3A6sH'")
(6, "'सूर्य ग्रहण 2022 : कुछ ऐसा था साल के आखरी सूर्य ग्रहण का नज़ारा\\n\\n#solareclipse #solarecl